# Time Series Project Notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_process import ArmaProcess
import statsmodels.api as sm
import statsmodels.formula.api as smf

import sktime
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.arima import AutoARIMA

from sklearn.metrics import mean_squared_error

from prophet import Prophet

import missingno as msno
import seaborn as sns
import warnings

from scipy.fft import fft, fftfreq

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

## 3. Understanding basic concepts in Time Series



### 3.1 From White Noise to ACF plots

1. Simulate a white noise for 200 observations and plot it

In [ ]:
np.random.seed(1234)
# generate white noise for 200 observations and plot it
white_noise = np.random.normal(0, 1, 200)
plt.plot(white_noise)
plt.show()

2. Create a white noise with mean = 4 and sd = 2. Then use the arima function to estimate the parameters

In [ ]:
# generate a white noise with mean 4 and sd 2 then use the arima function to estimate the parameters
white_noise = np.random.normal(4, 2, 200)
ARIMA(white_noise, order=(1, 0, 0)).fit().summary()

3. Explain with your own words what a random walk is (minimum 100 words). Explain with your own words what stationarity means for a time series.

*A random walk is a time series where the next value is dependent on the previous value. A white noise is a time series where the next value is independent of the previous value.*

*A stationary time series is a time series where the mean, variance and autocorrelation are constant over time.*

4. Create a random walk series, plot it, calculate the first difference series and plot it

In [ ]:
random_walk = np.cumsum(np.random.normal(0, 1, 200))
plt.plot(random_walk)
plt.plot(np.diff(random_walk))
plt.show()

5. Generate WN drift data, convert it to a random walk and plot it

In [ ]:
white_noise_drift = np.cumsum(np.random.normal(0, 1, 200)) + (np.arange(200) / 10)
plt.plot(white_noise_drift)
plt.show()

Plot the ACF on the white noise.

In [ ]:
plot_acf(white_noise, lags=20)
plt.show()

What’s the characteristic of a white noise ACF ?

*The ACF of a white noise is 0 for all lags. Moreover, they are never statistically significant.*

Perform a Ljung-Box Test. Command : Box.test in R, ljung is an option.

In [ ]:
acorr_ljungbox(white_noise, lags=20)

### 3.2 ARMA models

1. Use the arima.sim function (or Python equivalent) to generate time series based on the autoregressive model, with slopes comprised between -1 and 1.

In [ ]:
arima_sim = ArmaProcess(ar = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(arima_sim)
plt.show()

What do you observe ?

*The ACF of the random walk is not 0 for all lags. Moreover, they are statistically significant. also, the lags are positively correlated.*

Plot them, and the acf functions along with it.

In [ ]:
plot_acf(arima_sim, lags=20).show()

Do the same with the moving average model. What do you observe ?

In [ ]:
arima_sim = ArmaProcess(ma = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(arima_sim)
plt.show()

In [ ]:
plot_acf(arima_sim, lags=20).show()

What do you observe ?

*The ACF of the MA simulation is not 0 for all lags. Moreover, they are statistically significant. also, the lags are negatively correlated.*

2. Contrast AR(1) and AR(2) models. How do they differ ?

In [ ]:
# AR(1)
arima_sim = ArmaProcess(ar = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(arima_sim)


# AR(2)
arima_sim = ArmaProcess(ar = [0.3, -0.25, 0.3]).generate_sample(nsample=200)
plt.plot(arima_sim)

plt.legend(['AR(1)', 'AR(2)'])
plt.show()


*They differ by how the lags are correlated. The AR(2) model is more correlated to its 2 previous lags rather than the AR(1) model which is more correlated to only its first lag.*

3. What is the difference between an Autocorrelation Function and a Partial autocorrelation Function ? (Min. 150 words)

*The ACF is the correlation between the time series and its lags. The PACF is the correlation between the time series and its lags, while controlling for the effect of the intermediate lags. The PACF is a better indicator of the order of the AR model. The PACF of an AR(1) model is 0 for all lags after the first lag. The PACF of an AR(2) model is 0 for all lags after the second lag.*


4. Plot the ACF and the PACF of an AR, a MA, and ARMA models.

In [ ]:

ar_sim = ArmaProcess(ar = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(arima_sim)

ma_sim = ArmaProcess(ma = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(ma_sim)

arma_sim = ArmaProcess(ar = [0.3, -0.25], ma = [0.3, -0.25]).generate_sample(nsample=200)
plt.plot(arma_sim)

plt.legend(['AR', 'MA', 'ARMA'])
plt.show()

plot_acf(ar_sim, lags=20).show()
plot_acf(ma_sim, lags=20).show()
plot_acf(arma_sim, lags=20).show()


*The main difference between the ACF and the PACF is that the PACF is a better indicator of the order of the AR model. The PACF of an AR(1) model is 0 for all lags after the first lag. The PACF of an AR(2) model is 0 for all lags after the second lag.*

5. Write the equation of an ARMA model

$$X_t = \mu + \epsilon_t + \sum_{i=1}^p \phi_i X_{t-i} + \sum_{i=1}^q \theta_i \epsilon_{t-i}$$

6. What are the main differences between AIC and BIC criteria, conceptually speaking ? Elaborate (Min. 100 words).

The AIC criterion is the log-likelihood of the model plus a penalty term. The BIC criterion is the log-likelihood of the model plus a penalty term. The penalty term is higher for the BIC criterion. The AIC criterion is more suitable for small sample sizes, while the BIC criterion is more suitable for large sample sizes.


## 4. Forecasting competition on Kaggle


### 4.2 Exercises

1. Choose from the sales list the item, whose id is 20949.

In [ ]:
# Read the kaggle dataset
sales_train = pd.read_csv('Kaggle dataset/sales_train.csv')
sales_train["date"] = pd.to_datetime(sales_train["date"], format="%d.%m.%Y")

sales_train

In [ ]:
# Choose item #20949 in sales_train
sales_train_item20949 = sales_train[sales_train["item_id"] == 20949]
sales_train_item20949

2. Create a time series of the Kaggle dataset, but with a lag 1

In [ ]:
sales_train_item20949_lag1 = sales_train_item20949.shift(1)
sales_train_item20949_lag1

3. cbind the two datasets, and look at them using the head command

In [ ]:
to_merge = sales_train_item20949_lag1.iloc[1: , :].append([None])
merged_with_lag = pd.concat([sales_train_item20949, to_merge], axis=1).iloc[:-1, :].drop(0, axis=1)
merged_with_lag

4. use the cor function to look at the datasets, and then the acf function,with a lag 1. Plot the acf with different logs to see what happens.

In [ ]:
sns.heatmap(sales_train_item20949.corr(), annot=True)
sales_train_item20949["item_cnt_day"].plot()
plt.show()
plot_acf(sales_train_item20949["item_cnt_day"]).show()

In [ ]:
# Set index to date
sales_train_item20949 = sales_train_item20949.set_index("date")

5. Fit and plot an auto-regressive model to the time series.

In [ ]:
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(1, 0, 0))
model_fit = model.fit()
model_fit.summary()

In [ ]:
model_fit.plot_diagnostics()
plt.show()

6. What are the intercept and the innovation variance (sigma2) estimate ? What do these parameters mean ?

The intercept is the mean of the time series and the innovation variance is the variance of the error term.

7. Predict the sales for a month after the end of the training dataset.

In [ ]:
# Use the predict function, and the forecast function.
# Plot the results.
X = sales_train_item20949["item_cnt_day"].values
ARIMA_model = ARIMA(X, order=(3, 0, 0))
ARIMA_model_fit = ARIMA_model.fit()
predictions = ARIMA_model_fit.forecast(steps=700)

plt.plot(predictions, color='red')
plt.show()

8. Fit and plot a moving average model, and print the estimates

In [ ]:
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(0, 0, 1))
model_fit = model.fit()
model_fit.summary()

9. Make a 1 to 15 steps forecast, and plot the 95 percent confidence intervals

In [ ]:
X = sales_train_item20949["item_cnt_day"].values
ARIMA_model = ARIMA(X, order=(0, 0, 1))
ARIMA_model_fit = ARIMA_model.fit()
predictions = ARIMA_model_fit.get_forecast(steps=15)
confidence_interval = pd.DataFrame(predictions.conf_int(alpha = 0.05))

plt.plot(predictions.predicted_mean, color='red')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='red', alpha=.25)
plt.show()

10. Compare the goodness of fit of AR and MA models through AIC and BIC criteria

In [ ]:
# AR(1)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(1, 0, 0))
model_fit = model.fit()
model_fit.summary()


In [ ]:
# MA(1)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(0, 0, 1))
model_fit = model.fit()
model_fit.summary()

11. What model performs the best according to you ?

The best model in this case according to AIC and BIC is the AR model, because lower is better.

12. Make the ACF plots for various lags

In [ ]:
plot_acf(sales_train_item20949["item_cnt_day"], lags=20).show()

13. Use the residual analysis graphics of the Sarima function to check whether there are patterns in the residuals

In [ ]:
# Fit a SARIMA model
model = SARIMAX(sales_train_item20949["item_cnt_day"], order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))
model_fit = model.fit(disp=False)
model_fit.plot_diagnostics().show()

14. Do you see any patterns in the residuals for the diverse models you have implemented ?

No, the residuals are white noise.

15. How should Q-Q plot look like when the model is a good fit ?

The points of the Q-Q plot should all follow the line, in this case, the points follow the line quite well.

16. Fit various models (ARMA(1,1) - ARMA(2,1) - ARIMA(1,1,1) - ARIMA(1,1,0) to the Kaggle time series, and plot the t-table, check diagnostics


In [ ]:
# ARMA(1,1)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(1, 0, 1))
model_fit = model.fit()
model_fit.plot_diagnostics().show()
model_fit.summary()

In [ ]:
# ARMA(2,1)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(2, 0, 1))
model_fit = model.fit()
model_fit.plot_diagnostics().show()
model_fit.summary()


In [ ]:

# ARIMA(1,1,1)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(1, 1, 1))
model_fit = model.fit()
model_fit.plot_diagnostics().show()
model_fit.summary()


In [ ]:

# ARIMA(1,1,0)
model = ARIMA(sales_train_item20949["item_cnt_day"], order=(1, 1, 0))
model_fit = model.fit()
model_fit.plot_diagnostics().show()
model_fit.summary()

17. Fit a seasonal model to the Kaggle dataset. Fit ACF models with the relevant lags. Example : SARIMA(2,1,0,1,0,0,12). What are the conceptual differences between an ARIMA and a SARIMA, from the mathematical point of view ? Then, play with the parameters of the seasonnal component of the model (increase them), and see how forecasting is affected.

In [ ]:
# SARIMA(2,1,0,1,0,0,12)
model = SARIMAX(sales_train_item20949["item_cnt_day"], order=(2, 1, 0), seasonal_order=(1, 0, 0, 12))
model_fit = model.fit()
print(model_fit.summary())
model_fit.plot_diagnostics()

*The difference between ARIMA and SARIMA is that the ARIMA model is a linear model, while the SARIMA model is a non-linear model. The SARIMA model is a linear model with a seasonal component.*

18. Present a relevant sample of Ljung-Box tests, and explain how they can be used to assess your models


In [ ]:
acorr_ljungbox(model_fit.resid, lags=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], boxpierce=True)

*The Ljung-Box test is a test for autocorrelation. The null hypothesis is that the residuals are white noise. The p-value is the probability of rejecting the null hypothesis. If the p-value is lower than 0.05, we reject the null hypothesis and conclude that the residuals are not white noise.*

19. Use your favourite model to forecast the time series on the available test datasets


In [ ]:
# Fit a SARIMA model
model = SARIMAX(sales_train_item20949["item_cnt_day"], order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))
model_fit = model.fit(disp=False)
model_fit.plot_diagnostics()

# Make predictions and plot them using model_fit
predictions = model_fit.get_forecast(steps=700)
sarimax_predictions = predictions.predicted_mean
confidence_interval = pd.DataFrame(predictions.conf_int(alpha = 0.05))

plt.plot(predictions.predicted_mean, color='red')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='red', alpha=.25)
plt.show()


20. Perform a naive forecast based on the forecast package, to predict the sales for the training datasets

In [ ]:
sales_train_item20949

In [ ]:
sales_train_item20949.reset_index(inplace=True)
# Fit a naive model
naive_model = NaiveForecaster()
naive_model.fit(y = sales_train_item20949["item_cnt_day"], X = sales_train_item20949["date"])

# Make predictions and plot them using model_fit
predictions = naive_model.predict(fh=list(range(30)))
naive_predictions = predictions

plt.plot(sales_train_item20949["item_cnt_day"][-30:], color='blue')
plt.plot(predictions, color='red')
plt.show()

21. Remind us to what correspond the two shades of blue in the confidence intervals

*The two shades of blue correspond to the 95% confidence interval.*

22. Use the accuracy command to compute the RMSE statistics of your favourite models. What does RMSE mean ? What is the difference with MAE ? Why is it usually preferred to MSE ?

In [ ]:
sarimax_predictions[-10:]

In [ ]:
# Compute the accuracy of the naive model, and compare it to the accuracy of the SARIMA model

# Compute the accuracy of the naive model

print("Naive model accuracy : ", mean_squared_error(sales_train_item20949["item_cnt_day"][-30:], naive_predictions))

# Compute the accuracy of the SARIMA model

print("SARIMA model accuracy : ", mean_squared_error(sales_train_item20949["item_cnt_day"][-30:], sarimax_predictions[-30:]))

*From this we can conclude that a simple forecasting method is less accurate than a more complex one on the last month.*

*RMSE is the root mean squared error. It is the square root of the mean of the squared errors. The difference with MAE is that the RMSE is more sensitive to outliers. It is usually preferred to MSE because it is easier to interpret.*

23. Write a command that only returns the mean absolute percentage error

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

24. Compute cross-validated errors for up to a week ahead, with a naive forecast approach

In [ ]:
# Compute cross-validated errors for up to a week ahead, with a naive forecast approach

naive_model = NaiveForecaster(strategy="mean", sp=12)
naive_model.fit(y = sales_train_item20949["item_cnt_day"], X = sales_train_item20949["date"])
naive_predictions = naive_model.predict(fh=list(range(30)))

print("Naive model accuracy : ", mean_absolute_percentage_error(sales_train_item20949["item_cnt_day"][-30:], naive_predictions))


25. Use Auto-ARIMA to fit a model to your data. How does Auto-ARIMA work ?

In [ ]:
sales_datetime = sales_train_item20949.groupby("date").sum().reset_index()
sales_datetime = sales_datetime[["date", "item_cnt_day"]]
sales_datetime = sales_datetime.set_index("date")
sales_datetime = sales_datetime.asfreq('D')
sales_datetime = sales_datetime.fillna(0)
sales_datetime["item_cnt_day"] = sales_datetime["item_cnt_day"].astype(int)
sales_datetime

In [ ]:
# Use Auto-ARIMA to fit a model to your data. How does Auto-ARIMA work ?

# Fit an auto-ARIMA model
auto_arima_model = AutoARIMA(seasonal=True)
auto_arima_model.fit(sales_datetime["item_cnt_day"])

# Make predictions and plot them using model_fit
predictions = auto_arima_model.predict(fh=list(range(30)))
auto_arima_predictions = predictions

plt.plot(predictions, color='red')
plt.show()

*Auto-arima works by trying different combinations of parameters and choosing the one that minimizes the AIC criterion.*

## 5. Forecasting with Prophet

In this part we are going to focus on applying prophet to a dataset heavily influenced by human activity: crimes in Chicago.

In [ ]:
# import crimes data
crimes = pd.read_csv("crimes.csv")
crimes

In [ ]:
assaults = crimes[crimes["Primary Type"] == "ASSAULT"]
assaults.Date = pd.to_datetime(assaults.Date, format="%m/%d/%Y %I:%M:%S %p")
assaults["Date day"] = assaults.Date.dt.date
assaults

In [ ]:
# make predictions of the assaults with prophet
assaults_prophet = assaults.groupby("Date day").size().reset_index()
assaults_prophet.columns = ["ds", "y"]
assaults_prophet.ds = pd.to_datetime(assaults_prophet.ds)
assaults_prophet

In [ ]:
# Fit a prophet model
assaults_prophet_model = Prophet()
assaults_prophet_model.fit(assaults_prophet)

# Make predictions and plot them using model_fit
predictions = assaults_prophet_model.make_future_dataframe(periods=365)
predictions = assaults_prophet_model.predict(predictions)
assaults_prophet_predictions = predictions

assaults_prophet_model.plot(predictions, figsize=(7,5))
plt.show()

In [ ]:
assaults_prophet_model.plot_components(predictions, figsize=(7,5))
plt.show()

In [ ]:
# Make the same prediction using a SARIMA

# Fit a SARIMA model using AutoARIMA
model = AutoARIMA(seasonal=True)
model_fit = model.fit(assaults_prophet["y"])

In [ ]:
forecast = model_fit.predict(fh=list(range(365)))
conf_int = model_fit.predict_interval(fh=list(range(365)))
sarima_predictions = pd.DataFrame(forecast, columns=["yhat"])
sarima_predictions["yhat_lower"] = conf_int.iloc[:, 0]
sarima_predictions["yhat_upper"] = conf_int.iloc[:, 1]
sarima_predictions.index = pd.date_range(start=assaults_prophet.ds.max(), periods=365)
sarima_predictions.ds = pd.date_range(start=assaults_prophet.ds.max(), periods=365)


# Plot the predictions
plt.plot(assaults_prophet.set_index("ds")["y"], color='blue')
plt.plot(sarima_predictions["yhat"], color='red')
# plt.fill_between(sarima_predictions.ds, sarima_predictions["yhat_lower"], sarima_predictions["yhat_upper"], color='red', alpha=.25)
plt.plot(sarima_predictions["yhat_lower"], color='red', alpha=.25)
plt.plot(sarima_predictions["yhat_upper"], color='red', alpha=.25)
plt.show()

## 6. Discrete Fourier Transform

In [ ]:
# Make f(t)=2+0.75×sin(3wt)+0.25×sin(7wt)+0.5×sin(10wt) into python

# Create a function that returns the value of f(t) for a given t
def f(t, w):
    dc = 1

    return dc + 0.75 * np.sin(3 * w * t) + 0.25 * np.sin(7 * w * t) + 0.5 * np.sin(10 * w * t)

# Plot f
t = np.linspace(0, np.pi, 1000)
plt.show()

In [ ]:
# Make f(t)=2+0.75×sin(3wt)+0.25×sin(7wt)+0.5×sin(10wt) into python

# Create a function that returns the value of f(t) for a given t
def f(t, w):
    dc = 1

    return dc + 0.75 * np.sin(3 * w * t * np.pi) + 0.25 * np.sin(7 * w * t * np.pi) + 0.5 * np.sin(10 * w * t * np.pi)

# Plot f
t = np.linspace(0, 1, 1000)
plt.plot(t, f(t, 2))
plt.show()

In [ ]:
# Make the following R script into python
# acq.freq <- 100                    # data acquisition (sample) frequency (Hz)
# time     <- 6                      # measuring time interval (seconds)
# ts       <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 
# f.0 <- 1/time

# dc.component <- 1
# component.freqs <- c(3,7,10)        # frequency of signal components (Hz)
# component.delay <- c(0,0,0)         # delay of signal components (radians)
# component.strength <- c(1.5,.5,.75) # strength of signal components

# f   <- function(t,w) { 
#   dc.component + 
#   sum( component.strength * sin(component.freqs*w*t + component.delay)) 
# }

# plot.fourier <- function(fourier.series, f.0, ts) {
#   w <- 2*pi*f.0
#   trajectory <- sapply(ts, function(t) fourier.series(t,w))
#   plot(ts, trajectory, type="l", xlab="time", ylab="f(t)"); abline(h=0,lty=3)
# }

# plot.fourier(f,f.0,ts=ts)


# Python Version
acq_freq = 100                    # data acquisition (sample) frequency (Hz)
time = 6                      # measuring time interval (seconds)
ts = np.linspace(0, time, time*acq_freq) # vector of sampling time-points (s)
f_0 = 1/time

dc_component = 1
component_freqs = np.array([3,7,10])        # frequency of signal components (Hz)
component_delay = np.array([0,0,0])         # delay of signal components (radians)
component_strength = np.array([1.5,.5,.75]) # strength of signal components

def f(t, w):
    return dc_component + np.sum(component_strength * np.sin(component_freqs*w*t + component_delay))

def plot_fourier(fourier_series, f_0, ts):
    w = 2*np.pi*f_0
    trajectory = np.array([fourier_series(t, w) for t in ts])
    plt.plot(ts, trajectory)
    plt.axhline(y=dc_component, color='r', linestyle='dashed')

plot_fourier(f, f_0, ts=ts)


In [ ]:
# Convert this R function to python
# plot.frequency.spectrum <- function(X.k, xlimits=c(0,length(X.k))) {
#   plot.data  <- cbind(0:(length(X.k)-1), Mod(X.k))

#   # TODO: why this scaling is necessary?
#   plot.data[2:length(X.k),2] <- 2*plot.data[2:length(X.k),2] 
  
#   plot(plot.data, t="h", lwd=2, main="", 
#        xlab="Frequency (Hz)", ylab="Strength", 
#        xlim=xlimits, ylim=c(0,max(Mod(plot.data[,2]))))
# }

def plot_frequency_spectrum(X_k, xlimits=None):
    if xlimits is None:
        xlimits = [0, len(X_k) - 1]
    plot_data = np.array([[i, np.abs(X_k[i])] for i in range(len(X_k))])
    plot_data[1:, 1] = 2 * plot_data[1:, 1]
    plt.bar(plot_data[:, 0], plot_data[:, 1])
    plt.xlim(xlimits)
    plt.ylim([0, np.max(plot_data[:, 1])])
    plt.show()

# Convert the rest of the R script
# w <- 2*pi*f.0
# trajectory <- sapply(ts, function(t) f(t,w))
# X.k <- fft(trajectory)                   # find all harmonics with fft()
# plot.frequency.spectrum(X.k, xlimits=c(0,20))

w = 2*np.pi*f_0
trajectory = np.array([f(t, w) for t in ts])

X_k = np.fft.fft(trajectory)
plot_frequency_spectrum(X_k, xlimits=[0, 20])

In [ ]:
# Convert function to python
# get.trajectory <- function(X.k,ts,acq.freq) {
  
#   N   <- length(ts)
#   i   <- complex(real = 0, imaginary = 1)
#   x.n <- rep(0,N)           # create vector to keep the trajectory
#   ks  <- 0:(length(X.k)-1)
  
#   for(n in 0:(N-1)) {       # compute each time point x_n based on freqs X.k
#     x.n[n+1] <- sum(X.k * exp(i*2*pi*ks*n/N)) / N
#   }
  
#   x.n * acq.freq 
# }

def get_trajectory(X_k, ts, acq_freq):
    N = len(ts)
    i = complex(0, 1)
    x_n = np.zeros(N)
    ks = np.arange(len(X_k))
    
    for n in range(N):
        x_n[n] = np.sum(X_k * np.exp(i*2*np.pi*ks*n/N)) / N

    return x_n * acq_freq

In [ ]:
# R to python
# x.n <- get.trajectory(X.k,ts,acq.freq) / acq.freq  # TODO: why the scaling?
# plot(ts,x.n, type="l"); abline(h=0,lty=3)
# points(ts,trajectory,col="red",type="l") # compare with original

x_n = get_trajectory(X_k, ts, acq_freq) / acq_freq
plt.plot(ts, x_n, color='blue')
plt.plot(ts, trajectory, color='red')
plt.show()

In [ ]:
# Convert R to python
# plot.harmonic <- function(Xk, i, ts, acq.freq, color="red") {
#   Xk.h <- rep(0,length(Xk))
#   Xk.h[i+1] <- Xk[i+1] # i-th harmonic
#   harmonic.trajectory <- get.trajectory(Xk.h, ts, acq.freq=acq.freq)
#   points(ts, harmonic.trajectory, type="l", col=color)
# }

def plot_harmonic(X_k, i, ts, acq_freq, color='red'):
    colors = ['red', 'green', 'blue', 'yellow', 'black', 'purple']
    if type(color) == int:
        color = colors[color % len(colors)]
    X_k_h = np.zeros(len(X_k))
    X_k_h[i] = X_k[i]
    harmonic_trajectory = get_trajectory(X_k_h, ts, acq_freq)
    plt.plot(ts, harmonic_trajectory, color=color)

In [ ]:
# Convert R to python
# plot.show <- function(trajectory, time=1, harmonics=-1, plot.freq=FALSE) {

#   acq.freq <- length(trajectory)/time      # data acquisition frequency (Hz)
#   ts  <- seq(0,time-1/acq.freq,1/acq.freq) # vector of sampling time-points (s) 
  
#   X.k <- fft(trajectory)
#   x.n <- get.trajectory(X.k,ts, acq.freq=acq.freq) / acq.freq
  
#   if (plot.freq)
#     plot.frequency.spectrum(X.k)
  
#   max.y <- ceiling(1.5*max(Mod(x.n)))
  
#   if (harmonics[1]==-1) {
#     min.y <- floor(min(Mod(x.n)))-1
#   } else {
#     min.y <- ceiling(-1.5*max(Mod(x.n)))
#   }
  
#   plot(ts,x.n, type="l",ylim=c(min.y,max.y))
#   abline(h=min.y:max.y,v=0:time,lty=3)
#   points(ts,trajectory,pch=19,col="red")  # the data points we know
  
#   if (harmonics[1]>-1) {
#     for(i in 0:length(harmonics)) {
#       plot.harmonic(X.k, harmonics[i], ts, acq.freq, color=i+1)
#     }
#   }
# }

def plot_show(trajectory, time=1, harmonics=[-1], plot_freq=False):
    acq_freq = len(trajectory) / time
    ts = np.arange(0, time, 1/acq_freq)
    
    X_k = np.fft.fft(trajectory)
    x_n = get_trajectory(X_k, ts, acq_freq) / acq_freq
    
    if plot_freq:
        plot_frequency_spectrum(X_k)

    max_y = np.ceil(1.5 * np.max(np.abs(x_n)))
    
    if harmonics[0] == -1:
        min_y = np.floor(np.min(np.abs(x_n))) - 1
    else:
        min_y = np.ceil(-1.5 * np.max(np.abs(x_n)))
    
    plt.plot(ts, x_n, color='blue')
    plt.axhline(y=min_y, color='red', linestyle='dashed')
    plt.axhline(y=max_y, color='red', linestyle='dashed')
    plt.axvline(x=0, color='red', linestyle='dashed')
    plt.axvline(x=time-1/acq_freq, color='red', linestyle='dashed')
    plt.scatter(ts, trajectory, color='red')

    if harmonics[0] > -1:
        for i in range(len(harmonics)):
            # fix the problem with the first harmonic
            if harmonics[i] != 0:
                plot_harmonic(X_k, harmonics[i], ts, acq_freq, color=i)
            else :
                plt.plot(ts, np.zeros(len(ts)), color="red")


In [ ]:
# R to python
# trajectory <- 4:1
# plot_show(trajectory, time=2)

trajectory = np.arange(4, 0, -1)
plot_show(trajectory, time=2)

In [ ]:
# R to python
# trajectory <- c(rep(1,5),rep(2,6),rep(3,7))
# plot.show(trajectory, time=2, harmonics=0:3, plot.freq=TRUE)

trajectory = np.concatenate((np.ones(5), np.ones(6)*2, np.ones(7)*3))
plot_show(trajectory, time=2, harmonics=[0, 1, 2, 3], plot_freq=True)

In [ ]:
# R to python
# trajectory <- c(1:5,2:6,3:7)
# plot.show(trajectory, time=1, harmonics=c(1,2))

trajectory = np.concatenate((np.arange(1, 6), np.arange(2, 7), np.arange(3, 8)))
plot_show(trajectory, time=1, harmonics=[1, 2])

In [ ]:
# R to python
# set.seed(101)
# acq.freq <- 200
# time     <- 1
# w        <- 2*pi/time
# ts       <- seq(0,time,1/acq.freq)
# trajectory <- 3*rnorm(101) + 3*sin(3*w*ts)
# plot(trajectory, type="l")

np.random.seed(101)
acq_freq = 200
time = 1
w = 2 * np.pi / time
ts = np.arange(0, time, 1/acq_freq)
trajectory = 3 * np.random.randn(acq_freq) + 3 * np.sin(3 * w * ts)
plt.plot(trajectory, color='blue')
plt.show()

In [ ]:
# R to python
# X.k <- fft(trajectory)
# plot.frequency.spectrum(X.k,xlimits=c(0,acq.freq/2))

X_k = np.fft.fft(trajectory)
plot_frequency_spectrum(X_k, xlimits=[0, acq_freq/2])

In [ ]:
trajectory = trajectory + 25*ts
plt.plot(trajectory, color='blue')
plt.show()

In [ ]:
trend = smf.ols('trajectory ~ ts', data={'trajectory': trajectory, 'ts': ts}).fit()
detrended_trajectory = trend.resid
plt.plot(detrended_trajectory, color='blue')
plt.show()

## From shapelet mining to discord detection

Explain in 150 to 250 words what computing the matrix profile means. How
does it work ?

Computing the matrix profile means using a mathematical algorithm to identify patterns and trends in time series data. This is accomplished by creating a matrix from the time series data, with each column representing a different time point and each row representing a different time series.

The algorithm then calculates the similarity between each pair of columns in the matrix, and uses this information to identify patterns and trends in the data. This can include identifying "discordant" data points, which are points that do not fit the overall pattern of the data, and creating a "profile" of the data, which is a summary of the overall pattern of the data.

The matrix profile algorithm is useful for quickly and easily identifying patterns and trends in time series data, without having to manually inspect every time point individually. This can help data scientists to better understand and interpret the data, and to identify potential outliers or issues with the data.

### 7.1 Motif detection


#### What are the main parameters that you can play with during the motif detectionphase ? What happens when you change their value ? Elaborate (between 150 to 250 words).

During the motif detection phase, there are several main parameters that can be adjusted to change the behavior of the algorithm. These include:

1. The length of the motif: This parameter determines the length of the motifs that the algorithm will be searching for in the data. Increasing the length of the motif will typically result in more specific and accurate results, but will also require more computational resources. Changing this parameter will affect the granularity of the motifs that are identified, with longer motifs being more specific and shorter motifs being more general.
1. The distance metric: This parameter determines the mathematical formula that will be used to calculate the similarity between different time series. Different distance metrics will result in different results, and choosing the right metric can be critical for accurately identifying motifs in the data. Changing this parameter will affect the results of the motif detection phase, with different metrics producing different sets of motifs.
1. The number of motifs: This parameter determines the number of motifs that the algorithm will be searching for in the data. Increasing the number of motifs will typically result in more detailed and accurate results, but will also require more computational resources. Changing this parameter will affect the number of motifs that are identified, with more motifs resulting in a more detailed analysis of the data.
1. The minimum support: This parameter determines the minimum number of occurrences that a motif must have in order to be considered significant. Increasing the minimum support will typically result in more specific and accurate results, but will also reduce the number of motifs that are identified. Changing this parameter will affect the number of motifs that are identified, with higher minimum support levels resulting in fewer, but more significant, motifs.

Overall, these parameters can be adjusted to change the behavior of the algorithm and to fine-tune the results of the motif detection phase. By carefully adjusting these parameters, analysts can optimize the algorithm for their specific data and analysis goals.

#### Using the code provided by the instructor, apply it to the Motif detection dataset to identify a set of motifs, that you will highlight using colours.

Use the code given by the instructor.

#### Part 1: Synthetic Data

In [ ]:
from matrixprofile_ts import *

In [ ]:
data = pd.read_csv('rawdata.csv')
pattern = data.data.values

#Plot data
fig, ax1 = plt.subplots(figsize=(20,5))
ax1.plot(np.arange(len(pattern)),pattern, label="Synthetic Data")
legend = ax1.legend(loc='upper right')

In [ ]:
m=32
mp = matrixProfile.stomp(pattern,m)

In [ ]:
def plot_motifs(mtfs, labels, ax):

    colori = 0
    colors = 'rgbcm'
    for ms,l in zip(mtfs,labels):
        c =colors[colori % len(colors)]
        starts = list(ms)
        ends = [min(s + m,len(pattern)-1) for s in starts]
        ax.plot(starts, pattern[starts],  c +'o',  label=l)
        ax.plot(ends, pattern[ends],  c +'o', markerfacecolor='none')
        for nn in ms:
            ax.plot(range(nn,nn+m),pattern[nn:nn+m], c , linewidth=2)
        colori += 1

    ax.plot(pattern, 'k', linewidth=1, label="data")
    ax.legend()

In [ ]:
mtfs ,motif_d  = motifs.motifs(pattern, mp, max_motifs=10)

In [ ]:
#Append np.nan to Matrix profile to enable plotting against raw data
mp_adj = np.append(mp[0],np.zeros(m-1)+np.nan)

#Plot the signal data
fig, (ax1, ax2, ax3) = plt.subplots(3,1,sharex=True,figsize=(20,10))
ax1.plot(np.arange(len(pattern)),pattern, label="Synthetic Data")
ax1.set_ylabel('Signal', size=22)

#Plot the Matrix Profile
ax2.plot(np.arange(len(mp_adj)),mp_adj, label="Matrix Profile", color='red')
ax2.set_ylabel('Matrix Profile', size=22)

#Plot the Motifs
plot_motifs(mtfs, [f"{md:.3f}" for md in motif_d], ax3)
ax3.set_ylabel('Motifs', size=22)
#plt.xlim((0,100))
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1,sharex=True,figsize=(20,10))


mtfs ,motif_d  = motifs.motifs(pattern, mp, max_motifs=5, n_neighbors=4)
plot_motifs(mtfs, [f"{md:.3f}" for md in motif_d], ax1)
ax1.set_ylabel('4 Neigbhors', size=22)

mtfs ,motif_d  = motifs.motifs(pattern, mp, max_motifs=5, radius=10)
plot_motifs(mtfs, [f"{md:.3f}" for md in motif_d], ax2)
ax2.set_ylabel('Radius = 10', size=22)

mtfs ,motif_d  = motifs.motifs(pattern, mp, max_motifs=5, ex_zone=2*m)
plot_motifs(mtfs, [f"{md:.3f}" for md in motif_d], ax3)
ax3.set_ylabel('Exclude 2*m', size=22)
plt.show()

#### Part 2: NYC Taxis

In [ ]:
import matrixprofile as mp

# ignore matplotlib warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

In [ ]:
dataset = pd.read_csv('nyc_taxi.csv', parse_dates=["timestamp"]).rename(columns={'timestamp': 'datetime', 'value': 'data'})
dataset


In [ ]:
plt.figure(figsize=(15,3))
plt.plot(dataset['datetime'], dataset['data'])
plt.title('NYC Taxi Passenger Counts')
plt.ylabel('Passenger Count')
plt.xlabel('Datetime')
plt.tight_layout()
plt.show()

#### Imagine applications of motif detection in 5 different activity sectors (health, etc.). Min. 200 words.

There are many potential applications of motif detection in a variety of different activity sectors. Some examples include:

1. Health: Motif detection could be used to identify patterns and trends in medical data, such as heart rate or blood pressure readings. This could help doctors to identify potential health issues, such as arrhythmias or hypertension, and to monitor patients for potential changes in their health status.
1. Finance: Motif detection could be used to identify patterns and trends in financial data, such as stock prices or currency exchange rates. This could help investors to identify potential opportunities, such as buying low and selling high, and to avoid potential risks, such as market crashes or fraud.
1. Agriculture: Motif detection could be used to identify patterns and trends in agricultural data, such as crop yields or weather patterns. This could help farmers to optimize their operations, such as by planting at the right times and using the right fertilizers, and to adapt to changing conditions, such as droughts or pests.
1. Energy: Motif detection could be used to identify patterns and trends in energy data, such as electricity usage or renewable energy generation. This could help utilities to optimize their operations, such as by balancing supply and demand, and to adapt to changing conditions, such as increased demand or new technologies.
1. Education: Motif detection could be used to identify patterns and trends in educational data, such as student performance or teacher effectiveness. This could help educators to identify potential issues, such as low test scores or high dropout rates, and to develop strategies for improvement, such as targeted interventions or professional development.


Overall, motif detection has many potential applications in a wide range of activity sectors, and can be a valuable tool for identifying patterns and trends in data. By using motif detection, organizations in these sectors can better understand their data and use it to make informed decisions and take effective actions.

### 7.2 Discord detection

#### Based on the code provided by the instructor, compute the matrix profile for New York City taxi dataset and provide a list of five dates between 2014 and 2015 for which a discord was detected. Interpret these discords after an online search, with a different interpretation for each date.

#### Use the matrix profile technique again to find discords in an ECG. What could be the different applications of such a technique ?

The matrix profile technique could be used to find discords in an ECG (electrocardiogram) by creating a matrix from the ECG data, with each column representing a different time point and each row representing a different ECG waveform. The algorithm would then calculate the similarity between each pair of columns in the matrix, and use this information to identify discordant data points, which are points that do not fit the overall pattern of the data.

There are several potential applications for this technique, including:

1. Identifying potential health issues: By identifying discordant data points in an ECG, the matrix profile technique could be used to identify potential health issues, such as arrhythmias or heart attacks. This could be useful for doctors and other healthcare professionals, who could use the information to monitor patients and to take appropriate actions.
1. Detecting abnormalities in ECG data: The matrix profile technique could also be used to detect abnormalities in ECG data, such as spikes or dips in the waveform. This could be useful for researchers and other scientists, who could use the information to study the underlying causes of these abnormalities and to develop potential treatments.
1. Comparing ECG data to normal patterns: The matrix profile technique could also be used to compare ECG data to normal patterns, such as the average ECG waveform for a given age or gender. This could be useful for doctors and other healthcare professionals, who could use the information to assess the health of their patients and to identify potential issues.

Overall, the matrix profile technique could be a valuable tool for identifying and analyzing patterns in ECG data, and could have many different applications in the field of healthcare and medical research. By using the technique, doctors and other healthcare professionals could better understand their patients' ECG data, and use it to make more informed decisions and take more effective actions.

## Bonus: Mel Spectrogram


In [ ]:
import librosa
import librosa.display

filename = 'Haunting_song_of_humpback_whales.wav'
y, sr = librosa.load(filename)
# trim silent edges
whale_song, _ = librosa.effects.trim(y)
librosa.display.waveshow(whale_song, sr=sr)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_fft = 2048
D = np.abs(librosa.stft(whale_song[:n_fft], n_fft=n_fft, hop_length=n_fft+1))
plt.plot(D)
plt.show()

In [ ]:
hop_length = 512
D = np.abs(librosa.stft(whale_song, n_fft=n_fft,  hop_length=hop_length))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='linear')
plt.colorbar()
plt.show()

In [ ]:
DB = librosa.amplitude_to_db(D, ref=np.max)
librosa.display.specshow(DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.show()

In [ ]:
n_mels = 128
mel = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)

In [ ]:
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
librosa.display.specshow(mel, sr=sr, hop_length=hop_length, x_axis='linear')
plt.ylabel('Mel filter')
plt.colorbar()
plt.title('1. Our filter bank for converting from Hz to mels.')

plt.subplot(1, 3, 2)
mel_10 = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=10)
librosa.display.specshow(mel_10, sr=sr, hop_length=hop_length, x_axis='linear')
plt.ylabel('Mel filter')
plt.colorbar()
plt.title('2. Easier to see what is happening with only 10 mels.')

plt.subplot(1, 3, 3)
idxs_to_plot = [0, 9, 49, 99, 127]
for i in idxs_to_plot:
    plt.plot(mel[i])
plt.legend(labels=[f'{i+1}' for i in idxs_to_plot])
plt.title('3. Plotting some triangular filters separately.')

plt.tight_layout()

In [ ]:
plt.plot(D[:, 1])
plt.plot(mel.dot(D[:, 1]))
plt.legend(labels=['Hz', 'mel'])
plt.title('One sampled window for example, before and after converting to mel.')
plt.show()

In [ ]:
S = librosa.feature.melspectrogram(whale_song, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
S_DB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.show()